In [171]:
#Ejercicio 1

#declaraciones de importaciones que permiten acceder a diferentes funciones de bibliotecas externas
from docx import Document
from docx.shared import Inches, Pt, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_LINE_SPACING
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml.shared import OxmlElement, qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_TAB_ALIGNMENT, WD_TAB_LEADER
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_CELL_VERTICAL_ALIGNMENT
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.shared import Pt, RGBColor


def add_underline_space(paragraph, length):
    """Añade una línea subrayada para llenar espacios en blanco"""
    run = paragraph.add_run("_" * length)
    return run
def add_page_number(paragraph):
    """Añade un campo de número de página automático al párrafo"""
    # Crear elemento de campo
    fldChar1 = OxmlElement('w:fldChar')
    fldChar1.set(qn('w:fldCharType'), 'begin')
    
    # Crear propiedades del campo
    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')
    instrText.text = "PAGE"
    
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')
    
    # Crear elemento de texto para el número
    t = OxmlElement('w:t')
    t.text = "1"  # Texto que se verá en el editor, será reemplazado
    
    fldChar3 = OxmlElement('w:fldChar')
    fldChar3.set(qn('w:fldCharType'), 'end')
    
    # Ensamblar todos los elementos
    run = paragraph.add_run()
    r = run._r
    r.append(fldChar1)
    r.append(instrText)
    r.append(fldChar2)
    r.append(t)
    r.append(fldChar3)
    
    # Formatear el número de página
    run.font.name = 'Calibri'
    run.font.size = Pt(11)

def pag1():
    # Crear nuevo documento
    doc = Document()

    # Configurar página A4 (21.0 x 29.7 cm)
    section = doc.sections[0]
    section.page_width = Cm(21.0)
    section.page_height = Cm(29.7)
    
    # Configurar márgenes
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1)
        section.right_margin = Inches(1)

    # TABLA PRINCIPAL (1 columna, 3 filas - ahora con espacio para el logo)
    table = doc.add_table(rows=3, cols=1)
    table.autofit = False

    # CENTRAR LA TABLA EN LA PÁGINA
    table.alignment = WD_ALIGN_PARAGRAPH.CENTER
    # Configurar ancho de la tabla para que se vea centrada
    table.autofit = False
    for cell in table.columns[0].cells:
        cell.width = Inches(8)  # Ancho más amplio

    # Configurar ancho de columna
    for cell in table.columns[0].cells:
        cell.width = Inches(6)  # Ancho estándar para documento carta
    
    # Eliminar todos los bordes de la tabla
    no_borders = parse_xml(
        r'<w:tblBorders {}>'
        r'<w:top w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'<w:left w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'<w:bottom w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'<w:right w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'<w:insideH w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'<w:insideV w:val="none" w:sz="0" w:space="0" w:color="auto"/>'
        r'</w:tblBorders>'.format(nsdecls('w'))
    )
    table._tbl.tblPr.append(no_borders)
    
    # ELIMINAR ESPACIADO ENTRE CELDAS
    tbl_cell_spacing = OxmlElement('w:tblCellSpacing')
    tbl_cell_spacing.set(qn('w:w'), '0')
    tbl_cell_spacing.set(qn('w:type'), 'dxa')
    table._tbl.tblPr.append(tbl_cell_spacing)

    # Configurar cada fila para eliminar espaciado
    for row in table.rows:
        # Establecer altura mínima de fila
        tr = row._tr
        trPr = tr.get_or_add_trPr()
        trHeight = OxmlElement('w:trHeight')
        trHeight.set(qn('w:val'), '1')
        trHeight.set(qn('w:hRule'), 'auto')
        trPr.append(trHeight)
        
        # Configurar cada celda
        for cell in row.cells:
            # Eliminar márgenes de celda
            tc = cell._tc
            tcPr = tc.get_or_add_tcPr()
            tcMar = OxmlElement('w:tcMar')
            
            # Márgenes cero para todos los lados
            for margin in ['top', 'left', 'bottom', 'right']:
                mar = OxmlElement(f'w:{margin}')
                mar.set(qn('w:w'), '0')
                mar.set(qn('w:type'), 'dxa')
                tcMar.append(mar)
            
            tcPr.append(tcMar)
            
            # Eliminar espaciado de párrafos dentro de la celda
            for paragraph in cell.paragraphs:
                pPr = paragraph._element.get_or_add_pPr()
                spacing = OxmlElement('w:spacing')
                spacing.set(qn('w:before'), '0')
                spacing.set(qn('w:after'), '0')
                spacing.set(qn('w:line'), '240')
                spacing.set(qn('w:lineRule'), 'auto')
                pPr.append(spacing)

    
    # Fila 1: Logo y Nombre de compañía en la misma fila
    header_cell = table.rows[0].cells[0]
    header_cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER


    # Crear párrafo para el encabezado
    header_para = header_cell.paragraphs[0]
    
    # Añadir logo (alineado a la izquierda)
    try:
        # Añadir el logo primero
        logo_run = header_para.add_run()
        logo_run.add_picture('Parcial1_Candia_Zarco_Valeria_Ej1_img1.png', width=Inches(0.64))  # Tamaño más pequeño para compartir fila
        
        # Añadir tabulación para alinear el nombre de la compañía
        tab_stops = header_para.paragraph_format.tab_stops
        tab_stops.add_tab_stop(Inches(4), WD_TAB_ALIGNMENT.CENTER, WD_TAB_LEADER.SPACES)
        
        # Añadir tabulación y luego el nombre de la compañía con tamaño aumentado
        company_run = header_para.add_run("\tCompany Name Here\t")
        company_run.bold = True
        company_run.font.size = Pt(36)  # Tamaño 
        company_run.font.name = 'Times New Roman'

    except FileNotFoundError:
        print("Advertencia: No se encontró 'image.png'. Mostrando solo texto.")
        company_run = header_para.add_run("Company Name Here")
        company_run.bold = True
        company_run.font.size = Pt(36)
        header_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Fila 2: Dirección y otros (centrado)
    contact_cell = table.rows[1].cells[0]
    contact_cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    contact_para = contact_cell.paragraphs[0]
    contact_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    contact_para.paragraph_format.line_spacing = 1.15
    
    contact_run = contact_para.add_run(
        "[Street Address], [City, ST ZIP Code]\n"
        "[Phone: 555-555-55555] [Fax: 123-123-123456]\n"
        "[abc@example.com]"
    )
    contact_run.font.size = Pt(10)
    contact_run.font.name = 'Calibri'
    
    
    # FILA 3: TÍTULO DEL CONTRATO
    title_cell = table.rows[2].cells[0]
    title_cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    title_para = title_cell.paragraphs[0]
    title_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    title_run = title_para.add_run("Contract of Sale")
    title_run.font.size = Pt(37)
    title_run.font.name = 'Arial'
    title_run.bold = True

    # Ajustar altura de las filas
    table.rows[0].height = Inches(0.5)
    table.rows[1].height = Inches(0.5)
    table.rows[2].height = Inches(0.5)

    
    # Añadir contenido del contrato después de la tabla
    doc.add_paragraph("\n")  # Espacio adicional
    
    
    # Sección THIS AGREEMENT
    agreement_para = doc.add_paragraph()
    agreement_run1 = agreement_para.add_run("THIS AGREEMENT ")
    agreement_run1.bold = True
    agreement_run1.font.size = Pt(11)
    agreement_run1.font.name = 'Calibri'

    agreement_run2 = agreement_para.add_run("made this ________________ day of _______________________, 20________________, by and between")
    agreement_run2.font.size = Pt(10)
    agreement_run2.font.name = 'Calibri'
    agreement_run3 = agreement_para.add_run("" + "_" * 63 + " (First Party) and")
    agreement_run3.font.size = Pt(10) 
    agreement_run3.font.name = 'Calibri'
    agreement_run4 = agreement_para.add_run("\n" + "_" * 65 + " (Second Party).")
    agreement_run4.font.size = Pt(10) 
    agreement_run4.font.name = 'Calibri'

    
    # WITNESSETH
    witness_para = doc.add_paragraph()
    witness_run1 = witness_para.add_run("WITNESSETH: ")
    witness_run1.font.name = 'Calibri'
    witness_run1.font.size =Pt(12)
    witness_run1.bold = True
    witness_run2 = witness_para.add_run(
        "That in consideration of the mutual covenants and agreements to be kept and "
        "performed on the part of said parties hereto, respectively as herein stated:"
    )
    witness_run2.font.name='Calibri'
    witness_run2.font.size=Pt(11)
    
    # Crear estilo personalizado para párrafos
    styles = doc.styles
    try:
        # Intentar crear el estilo
        roman_style = styles.add_style('RomanListPara', WD_STYLE_TYPE.PARAGRAPH)
        roman_style.base_style = styles['Normal']
    
        # Configurar fuente
        font = roman_style.font
        font.name = 'Calibri'
        font.size = Pt(11)
    
        # Configurar formato de párrafo para simular lista
        para_format = roman_style.paragraph_format
        para_format.left_indent = Inches(0.5)
        para_format.first_line_indent = Inches(-0.25)  # Sangría francesa
    
    except ValueError:
        # Si el estilo ya existe, usarlo
        roman_style = styles['RomanListPara']

    # Sección I con número romano
    section1_para = doc.add_paragraph(style='RomanListPara')
    roman_run = section1_para.add_run("I.")
    roman_run.bold = True
    text_run = section1_para.add_run("\t Said party of the first part covenants and agrees that it shall:")
    text_run.bold = True

    # Líneas en blanco subrayadas para sección I (4 líneas)
    for _ in range(4):
        line_para = doc.add_paragraph()
        line_para.paragraph_format.left_indent = Inches(0.6)
        add_underline_space(line_para, 100)
        line_para.paragraph_format.space_after = Pt(1.5)
    # Línea corta al final de la sección I
    short_line_para = doc.add_paragraph()
    short_line_para.paragraph_format.left_indent = Inches(0.6)
    add_underline_space(short_line_para, 30)
    short_line_para.paragraph_format.space_after = Pt(1.5)
    
    # Sección II con número romano
    section2_para = doc.add_paragraph(style='RomanListPara')
    roman_run = section2_para.add_run("II.")
    roman_run.bold = True
    text_run = section2_para.add_run("\t And said party of the second part covenants and agrees that it shall:")
    text_run.bold = True

    # Líneas en blanco subrayadas para sección II (4 líneas)
    for _ in range(4):
        line_para = doc.add_paragraph()
        line_para.paragraph_format.left_indent = Inches(0.6)
        add_underline_space(line_para, 100)
        line_para.paragraph_format.space_after = Pt(1.5)

    # Línea corta al final de la sección II
    short_line_para = doc.add_paragraph()
    short_line_para.paragraph_format.left_indent = Inches(0.6)
    add_underline_space(short_line_para, 30)
    short_line_para.paragraph_format.space_after = Pt(1.5)

    # Sección III con número romano
    section3_para = doc.add_paragraph(style='RomanListPara')
    roman_run = section3_para.add_run("III.")
    roman_run.bold = True
    text_run = section3_para.add_run("\tOther terms to be observed by and between the parties:")
    text_run.bold = True

    # Líneas en blanco subrayadas para sección III (4 líneas)
    for _ in range(4):
        line_para = doc.add_paragraph()
        line_para.paragraph_format.left_indent = Inches(0.6)
        add_underline_space(line_para, 100)
        line_para.paragraph_format.space_after = Pt(1.5)

    # Línea corta al final de la sección III
    short_line_para = doc.add_paragraph()
    short_line_para.paragraph_format.left_indent = Inches(0.6)
    add_underline_space(short_line_para, 30)
    short_line_para.paragraph_format.space_after = Pt(1.5)



    # Form of payment
    form_para = doc.add_paragraph()
    form_run = form_para.add_run("Form of payment")
    form_run.bold = True
    form_run.underline = True
    
    # Configurar el estilo de las viñetas
    estilo = doc.styles['List Bullet']
    fuente = estilo.font
    fuente.name = 'Calibri'
    fuente.size = Pt(11)
    
    # Agregar elementos de la lista con viñetas circulares
    opciones = [
        'Prepayment',
        'open account with remittance at specified date',
        'letter of credit',
        'payment by nominated acceptable third party such as parent company or guarantor.'
    ]
    
    # Agregar lista con viñetas automáticas
    for opcion in opciones:
        p = doc.add_paragraph(opcion, style='List Bullet')
        # Ajustar sangría para que coincida con tu ejemplo
        p.paragraph_format.left_indent = Inches(1.3)
        p.paragraph_format.first_line_indent = Inches(-0.25)
        p.paragraph_format.line_spacing_rule = WD_LINE_SPACING.EXACTLY
        p.paragraph_format.line_spacing = Pt(12)  # Puedes ajustar este valor
    
    # Configurar pie de página con numeración automática
    section = doc.sections[0]
    footer = section.footer
    
    # Limpiar párrafos existentes en el footer
    for p in footer.paragraphs:
        p.clear()
    
    # Crear párrafo centrado para el número
    footer_para = footer.add_paragraph()
    footer_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Añadir número de página automático
    add_page_number(footer_para)


    ##############################################################################################################
    #PAGINA 2
    ##############################################################################################################

    # Párrafo principal
    p1 = doc.add_paragraph()
    p1.paragraph_format.line_spacing = 0.9  # Interlineado reducido
    run1 = p1.add_run("Always including a schedule of documents which will be required to authenticate payment (such as are usually required for letters of credit); e.g.")
    run1.font.name = 'Calibri'
    run1.font.size = Pt(11)

    # Función para crear numeración con letras
    def add_letter_list_paragraph(doc, letter, text_parts):
        p = doc.add_paragraph()

        # Reducir interlineado en los párrafos de lista
        p.paragraph_format.line_spacing = 1.0  # Interlineado reducido
        p.paragraph_format.space_before = Pt(0)
        p.paragraph_format.space_after = Pt(3)
        # Agregar la letra manualmente
        run_letter = p.add_run(f"{letter}. ")
        run_letter.font.name = 'Calibri'
        run_letter.font.size = Pt(11)
        
        # Agregar el resto del contenido
        for i, (text, is_bold) in enumerate(text_parts):
            run = p.add_run(text)
            run.bold = is_bold
            run.font.name = 'Calibri'
            run.font.size = Pt(11)
        
        # Aplicar indentación para simular lista
        p.paragraph_format.left_indent = Inches(0.5)
        p.paragraph_format.first_line_indent = Inches(-0.25)
        return p

    # a. Bill of Lading
    add_letter_list_paragraph(doc, 'a', [
        ("  Bill of Lading", False),
        (" – should contain details of consignor (if not shipped in name of seller), consignee, notify party, negotiable copies, clean or special clause bill, freight prepaid or payable at destination or by third party 'shipped' or 'received', throughbill, optional transhipment/destination port/s, etc.", False)
    ])

    # b. Insurance policy
    add_letter_list_paragraph(doc, 'b', [
        ("  Insurance policy or certificate of insurance", False),
        (", declaration/notification for purchasers open policy, required cover, e.g.", False)
    ])
    print("")
    # Agregar espacio ANTES de la tabla
    p_pre_tabla = doc.add_paragraph()
    p_pre_tabla.paragraph_format.space_after = Pt(6)  # Espacio de 12 puntos después (antes de la tabla)
    p_pre_tabla.paragraph_format.line_spacing = 0.5

    # Tabla con las cláusulas del seguro (sin bordes, centrada y compacta)
    table = doc.add_table(rows=7, cols=1)
    table.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Reducir el ancho de la tabla significativamente
    table.autofit = False
    table.allow_autofit = False
    table.columns[0].width = Inches(4.5)  # Ancho de columna más estrecho
    
    # Configurar la tabla para eliminar bordes y reducir espaciado al mínimo
    for row in table.rows:
        row.height = Inches(0.12)  # Altura de filas aún más pequeña
        for cell in row.cells:
            # Eliminar bordes
            tc = cell._tc
            tcPr = tc.get_or_add_tcPr()
            tcBorders = tcPr.first_child_found_in("w:tcBorders")
            if tcBorders is not None:
                tcPr.remove(tcBorders)
            
            # Reducir márgenes internos al mínimo
            cell.top_margin = Inches(0.01)
            cell.bottom_margin = Inches(0.01)
            cell.left_margin = Inches(0.02)   # Bordes laterales más pequeños
            cell.right_margin = Inches(0.02)  # Bordes laterales más pequeños

    # Contenido de las celdas
    clauses = [
        "Institute Cargo Clauses (A) 1.1.82",
        "Institute War Clauses 1.1.82", 
        "Institute Strikes Clauses (Cargo) 1.1.82",
        "Particular and General Average Cover",
        "Jetty Survey Clause if necessary",
        "Diversion Strike Expenses Clause if necessary",
        "Also place and procedure relating to surveys/claims for loss or damage, surveyor's name and address."
    ]

    for i, clause in enumerate(clauses):
        cell = table.rows[i].cells[0]
        paragraph = cell.paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        # Reducir espaciado entre párrafos dentro de la celda
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        paragraph.paragraph_format.line_spacing = 1.0
        
        run = paragraph.add_run(clause)
        run.italic = True
        run.font.name = 'Calibri'
        run.font.size = Pt(11)

    # Ajustar el ancho total de la tabla después de configurar todo
    tbl = table._tbl
    tblPr = tbl.tblPr
    tblW = tblPr.first_child_found_in("w:tblW")
    if tblW is not None:
        tblPr.remove(tblW)

    # Crear nuevo elemento de ancho de tabla
    tblW = parse_xml(r'<w:tblW {} w:w="5200" w:type="dxa"/>'.format(nsdecls('w')))
    tblPr.insert(0, tblW)

    # Agregar espacio DESPUÉS de la tabla
    p_post_tabla = doc.add_paragraph()
    p_post_tabla.paragraph_format.space_before = Pt(6)  # Espacio de 12 puntos antes (después de la tabla)
    p_post_tabla.paragraph_format.line_spacing = 0.5

    # c. Invoices
    add_letter_list_paragraph(doc, 'c', [
        ("  Invoices", False),
        (" – commercial, consular, specially endorsed or certified, etc.", False)
    ])

    # d. Certificates
    add_letter_list_paragraph(doc, 'd', [
        ("  Certificates of evidence required", False),
        (" – relating to weight, measure, quality, grading, health, origin, analysis, etc.", False)
    ])

    # e. Any other documentation
    add_letter_list_paragraph(doc, 'e', [
        ("  Any other documentation required by either party", False),
        (".", False)
    ])

    # f. Distribution
    add_letter_list_paragraph(doc, 'f', [
        ("  Distribution of documentation", False),
        (".", False)
    ])

    # Párrafo PURSUANT TO
    p_pursuant = doc.add_paragraph()
    p_pursuant.paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    # Reducir interlineado en el párrafo PURSUANT TO
    p_pursuant.paragraph_format.line_spacing = 1.0  # Interlineado reducido
    p_pursuant.paragraph_format.space_before = Pt(6)
    p_pursuant.paragraph_format.space_after = Pt(6)
    run_p1 = p_pursuant.add_run("PURSUANT TO")
    run_p1.bold = True
    run_p1.font.name = 'Calibri'
    run_p1.font.size = Pt(10)
    
    run_p2 = p_pursuant.add_run(" the ")
    run_p2.font.name = 'Calibri'
    run_p2.font.size = Pt(10)
    
    run_p3 = p_pursuant.add_run("Funeral, Burial and Cremation Services Act")
    run_p3.italic = True
    run_p3.font.name = 'Calibri'
    run_p3.font.size = Pt(10)
    
    run_p4 = p_pursuant.add_run(", 2002 and ")
    run_p4.font.name = 'Calibri'
    run_p4.font.size = Pt(10)
    
    run_p5 = p_pursuant.add_run("Ontario Regulation 31/11")
    run_p5.italic = True
    run_p5.font.name = 'Calibri'
    run_p5.font.size = Pt(10)
    
    run_p6 = p_pursuant.add_run(" and all amendments thereto, the Interment Rights Holder (s) listed below have the right to direct/consent to the burial and memorialization associated with the Interment Rights in conjunction with the Town of Greater Napanee and the By-laws for ")
    run_p6.font.name = 'Calibri'
    run_p6.font.size = Pt(10)
    
    run_p7 = p_pursuant.add_run("The Napanee Cemetery Company, Riverside Cemetery & Riverview Cemetery. ")
    run_p7.italic = True
    run_p7.underline=True
    run_p7.font.name = 'Calibri'
    run_p7.font.size = Pt(10)
    
    run_p8 = p_pursuant.add_run(" This Contract is between the Purchaser and The Town of Greater Napanee, concerning Cemetery Interment Rights for the Recipient(s) as identified in this Contract. The Purchaser represents being legally authorized or charged with the responsibility for the Recipient(s) Cemetery Interment Rights and Services as specified in this Contract. ")
    run_p8.font.name = 'Calibri'
    run_p8.font.size = Pt(10)
    
    run_p9 = p_pursuant.add_run("This agreement will be enforceable to the benefit of")
    run_p9.bold = True
    run_p9.font.name = 'Calibri'
    run_p9.font.size = Pt(10)

    # Línea en blanco para la firma
    p_firma = doc.add_paragraph("_" * 40+ ".")
    p_firma.alignment = WD_ALIGN_PARAGRAPH.LEFT
    doc.add_paragraph("\n")  # Espacio adicional
    doc.add_paragraph("\n")  # Espacio adicional
    doc.add_paragraph("\n")  # Espacio adicional
    doc.add_paragraph("\n")  # Espacio adicional
    doc.add_paragraph("\n")  # Espacio adicional


    ################################################################################################
    #PAGINA 3
    ################################################################################################
    # === FUNCIONES AUXILIARES ===
    def set_cell_background(cell, fill_color):
        """Establece el color de fondo de una celda"""
        tcPr = cell._tc.get_or_add_tcPr()
        shading = OxmlElement('w:shd')
        shading.set(qn('w:fill'), fill_color)
        tcPr.append(shading)

    def add_table_borders(table):
        tbl = table._tbl
        for row in tbl.tr_lst:
            for cell in row.tc_lst:
                tcPr = cell.tcPr
                tcBorders = OxmlElement('w:tcBorders')
            
            # Crear bordes para todos los lados
            for border_name in ['top', 'left', 'bottom', 'right']:
                border = OxmlElement(f'w:{border_name}')
                border.set(qn('w:val'), 'single')
                border.set(qn('w:sz'), '4')
                border.set(qn('w:space'), '0')
                border.set(qn('w:color'), '000000')
                tcBorders.append(border)
            
            tcPr.append(tcBorders)
        
    def set_cell_format(cell, font_name='Calibri', font_size=11, bold=True, space_after=Pt(12)):
        """Configura el formato de texto y espaciado para una celda"""
        for paragraph in cell.paragraphs:
            paragraph.paragraph_format.space_after = Pt(18)  # Más espacio abajo
            paragraph.paragraph_format.space_before = Pt(6)  # Espacio arriba
            for run in paragraph.runs:
                run.font.name = font_name
                run.font.size = Pt(font_size)
                run.bold = bold

    # === TABLA 1: PURCHASER DETAILS ===
    # Crear tabla con título integrado
    table1 = doc.add_table(rows=10, cols=4)
    table1.style = 'Table Grid'
    
    # Configurar primera fila (título)
    title_cell = table1.cell(0, 0)
    title_cell.text = "Purchaser Details"
    title_cell.merge(table1.cell(0, 3))
    set_cell_background(title_cell, 'D9D9D9')  # Gris claro
    set_cell_format(title_cell)
    for paragraph in title_cell.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name='Calibri'
    
    # Configurar segunda fila (Name)
    table1.cell(1, 0).text = "Name"
    table1.cell(1, 1).merge(table1.cell(1, 3))
    set_cell_format(table1.cell(1, 0))
    set_cell_format(table1.cell(1, 1))
    
    # Tercera fila (Address)
    table1.cell(2, 0).text = "Address"
    table1.cell(2, 1).merge(table1.cell(2, 3))
    set_cell_format(table1.cell(2, 0))
    set_cell_format(table1.cell(2, 1))
    
    # Cuarta fila (Telephone y Email)
    table1.cell(3, 0).text = "Telephone"
    set_cell_format(table1.cell(3, 0))
    table1.cell(3, 2).text = "Email"
    set_cell_format(table1.cell(3, 2))
    
    # Quinta fila (Recipients instruction)
    recip_cell = table1.cell(4, 0).merge(table1.cell(4, 3))
    recip_cell.text = "Recipient(s): List all recipients and contact information below."
    set_cell_format(recip_cell)
    
    # Sexta fila (headers para recipients)
    table1.cell(5,0).text="Name"
    table1.cell(5,1).text="Contact Information"
    tresco_cell = table1.cell(5, 2).merge(table1.cell(5, 3))
    tresco_cell.text="Relationship to Recipient"
    set_cell_format(table1.cell(5, 0), bold=True)
    set_cell_format(table1.cell(5, 1), bold=True)
    set_cell_format(tresco_cell, bold=True)
    
    # Séptima fila (6 en base 0)
    tabla1fila6 = table1.cell(6, 2).merge(table1.cell(6, 3))
    tabla1fila6.paragraphs[0].paragraph_format.space_after = Pt(11)  # Más espacio abajo
    tabla1fila6.paragraphs[0].paragraph_format.space_before = Pt(6)

    # Octava fila (7 en base 0)
    tabla1fila7 = table1.cell(7, 2).merge(table1.cell(7, 3))
    tabla1fila7.paragraphs[0].paragraph_format.space_after = Pt(11)
    tabla1fila7.paragraphs[0].paragraph_format.space_before = Pt(6)

    # Novena fila (8 en base 0)
    tabla1fila8 = table1.cell(8, 2).merge(table1.cell(8, 3))
    tabla1fila8.paragraphs[0].paragraph_format.space_after = Pt(11)
    tabla1fila8.paragraphs[0].paragraph_format.space_before = Pt(6)

    # Décima fila (9 en base 0)
    tabla1fila9 = table1.cell(9, 2).merge(table1.cell(9, 3))
    tabla1fila9.paragraphs[0].paragraph_format.space_after = Pt(11)
    tabla1fila9.paragraphs[0].paragraph_format.space_before = Pt(6)
    # Aplicar bordes
    add_table_borders(table1)
    
    # Espacio entre tablas
    doc.add_paragraph()
    
    # === TABLA 2: INTERMENT RIGHTS DETAILS =============
    # Crear tabla con título integrado
    table2 = doc.add_table(rows=5, cols=8)
    table2.style = 'Table Grid'

    # FILA 1: Título "Interment Rights Details"
    title_cell2 = table2.cell(0, 0)
    title_cell2.text = "Interment Rights Details"
    set_cell_format(table2.cell(0, 0))
    # Fusionar todas las columnas de la primera fila
    for col in range(1, 8):
        title_cell2.merge(table2.cell(0, col))

    set_cell_background(title_cell2, 'D9D9D9')  # Gris claro
    for paragraph in title_cell2.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name = 'Calibri'

    # FILA 2: "Cemetery" con espacio para datos
    cemetery_cell = table2.cell(1, 0)
    cemetery_cell.text = "Cemetery"
    set_cell_format(cemetery_cell)
    # Fusionar las columnas restantes para el campo de datos
    data_cell = table2.cell(1, 1)
    for col in range(2, 8):
        data_cell.merge(table2.cell(1, col))

    for paragraph in cemetery_cell.paragraphs:
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name = 'Calibri'

    # FILA 3-4: Headers con sus espacios de datos correspondientes
    # No. of Graves (columna 0)
    nograves_cell = table2.cell(2, 0)
    nograves_cell.text = "No. of Graves"
    set_cell_format(table2.cell(2, 0))
    nograves_cell.merge(table2.cell(3, 0))  # Fusionar verticalmente

    # Section (columna 2)
    section_cell = table2.cell(2, 2)
    section_cell.text = "Section"
    section_cell.merge(table2.cell(3, 2))  # Fusionar verticalmente

    # Plot (columna 4)
    plot_cell = table2.cell(2, 4)
    plot_cell.text = "Plot"
    set_cell_format(table2.cell(2, 4))
    plot_cell.merge(table2.cell(3, 4))  # Fusionar verticalmente

    # Quadrant(s) (columna 6)  
    quadrant_cell = table2.cell(2, 6)
    quadrant_cell.text = "Quadrant(s)"
    set_cell_format(table2.cell(2, 6))
    quadrant_cell.merge(table2.cell(3, 6))  # Fusionar verticalmente

    # Aplicar formato a los headers
    for cell in [nograves_cell, section_cell, plot_cell, quadrant_cell]:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.bold = True
                run.font.size = Pt(11)
                run.font.name = 'Calibri'

    # Fusionar celdas de datos verticalmente para crear espacios
    # Datos para No. of Graves (columna 1)
    #table2.cell(2, 1).merge(table2.cell(3, 1))
    table2.rows[2].height = Pt(24)  # Fila superior
    table2.rows[3].height = Pt(24)  # Fila inferior
    # Datos para Section (columna 3) 
    #table2.cell(2, 3).merge(table2.cell(3, 3))

    # Datos para Plot (columna 5)
    #table2.cell(2, 5).merge(table2.cell(3, 5))

    # Datos para Quadrant(s) (columna 7)
    #table2.cell(2, 7).merge(table2.cell(3, 7))

    # FILA 5: "Notes" con espacio para datos
    notes_cell = table2.cell(4, 0)
    notes_cell.text = "Notes"
    set_cell_format(notes_cell)
    # Fusionar el resto de las columnas para el campo de notes
    notes_data_cell = table2.cell(4, 1)
    for col in range(2, 8):
        notes_data_cell.merge(table2.cell(4, col))

    for paragraph in notes_cell.paragraphs:
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name = 'Calibri'

    # Aplicar bordes
    add_table_borders(table2)
    # Espacio entre tablas
    doc.add_paragraph()
    
    # === TABLA 3: DECEASED INFORMATION ===
    # FILA 1: Título "Deceased Information (If applicable)"
    # Crear tabla con 4 filas y 3 columnas
    table3 = doc.add_table(rows=3, cols=4)
    table3.style = 'Table Grid'
    title_cell3 = table3.cell(0, 0)
    title_cell3.text = "Deceased Information (If applicable)"
    set_cell_format(title_cell3)
    # Fusionar todas las columnas de la primera fila
    for col in range(1, 4):
        title_cell3.merge(table3.cell(0, col))
    
    set_cell_background(title_cell3, 'D9D9D9')  # Gris claro
    for paragraph in title_cell3.paragraphs:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name = 'Calibri'
    
    # FILA 2: "Name" con espacio para datos
    name_cell = table3.cell(1, 0)
    name_cell.text = "Name"
    set_cell_format(name_cell)
    # Fusionar las columnas restantes para el campo de datos del nombre
    name_data_cell = table3.cell(1, 1)
    name_data_cell.merge(table3.cell(1, 3))
    
    for paragraph in name_cell.paragraphs:
        for run in paragraph.runs:
            run.bold = True
            run.font.size = Pt(11)
            run.font.name = 'Calibri'
    
    # FILA 3: "Place of Death" y "Date of Death" (headers)
    place_death_cell = table3.cell(2, 0)
    place_death_cell.text = "Place of Death"
    set_cell_format(place_death_cell)
    
    date_death_cell = table3.cell(2, 2)
    date_death_cell.text = "Date of Death"
    set_cell_format(date_death_cell)
    
    # Aplicar formato a los headers
    for cell in [place_death_cell, date_death_cell]:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.bold = True
                run.font.size = Pt(11)
                run.font.name = 'Calibri'
    
    # FILA 4: Espacios para datos de Place of Death y Date of Death
    # Las celdas (3,0), (3,1), y (3,2) quedan como espacios para datos
    # La celda (3,1) queda como espacio para datos de Place of Death
    # La celda (3,2) queda como espacio para datos de Date of Death
    
    # Aplicar bordes
    add_table_borders(table3)
    
    # Agregar espacio después de la tabla
    doc.add_paragraph("")

    # Guardar el documento
    doc.save('Parcial1_Candia_Ej1.docx')
    print("Documento creado exitosamente: 'Parcial1_Ej1_Candia.docx'")

# Ejecutar la función
pag1()


Documento creado exitosamente: 'Parcial1_Ej1_Candia.docx'
